In [ ]:
PurchaseOrderDetail = spark.read.load('abfss://root@azuredatalakelabs.dfs.core.windows.net/myfiles/PurchaseOrderDetail.csv'
             ,format='csv'
             ,header=True
)
display(PurchaseOrderDetail.limit(3))

In [ ]:
%%pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [14]:
%%pyspark
orderSchema = StructType([
    StructField("PurchaseOrderID", IntegerType()),
    StructField("PurchaseOrderDetailID", IntegerType()),
    StructField("DueDate", DateType()),
    StructField("OrderQty", IntegerType()),
    StructField("ProductID", IntegerType()),
    StructField("UnitPrice", DecimalType(12,2)),
    StructField("LineTotal", DecimalType(12,2)),
    StructField("ReceivedQty", DecimalType(12,2)),
    StructField("RejectedQty", DecimalType(12,2)),
    StructField("StockedQty", DecimalType(12,2)),
    StructField("ModifiedDate", DateType())
    ])

PurchaseOrderDetail = spark.read.load('abfss://root@azuredatalakelabs.dfs.core.windows.net/myfiles/PurchaseOrderDetail.csv'
             ,format='csv'
             ,schema=orderSchema
             ,header=True
)
display(PurchaseOrderDetail.limit(3))

In [9]:
PurchaseOrderDetail.createOrReplaceTempView("SalesOrderDetail")

In [46]:
from matplotlib import pyplot as plt

In [83]:
%%pyspark
SalesOrderDetail = spark.sql("""
    WITH SalesOrderDetail (DueDate,TotalPurchasePrice,TotalSalesPrice,ReceivedQty)
    AS 
    ( 
        SELECT DueDate
            ,SUM(UnitPrice) AS TotalPurchasePrice
            ,SUM(LineTotal) AS TotalSalesPrice
            ,COUNT(ReceivedQty) AS ReceivedQty
        FROM SalesOrderDetail
        WHERE YEAR(DueDate) ='2013'
          AND MONTH(DueDate) IN (1,2,3,4,5,6)
        GROUP BY DueDate
    )
    SELECT DueDate
        ,TotalPurchasePrice * ReceivedQty AS TotalPurchasePrice
        ,TotalSalesPrice
        ,TotalSalesPrice - TotalPurchasePrice AS TotalRevenue
        ,ReceivedQty
    FROM SalesOrderDetail
    ORDER BY DueDate
""").toPandas()

In [111]:
%%pyspark
from matplotlib import pyplot as plt

# clear the plot area
plt.clf()

# create a figure
figure = plt.figure(figsize=(12, 8))

# create a bar plot of total sales by month
plt.bar(x=SalesOrderDetail['DueDate'], height=SalesOrderDetail['TotalSalesPrice'], color='green')

# customize the chart
plt.title('2013 First & Second Quarter')
plt.xlabel('Sales Order Details')
plt.ylabel('Revenue by Total number of orders')
plt.grid(color='#95a5a6', linestyle='--')

# show the plot area
plt.show()

In [110]:

plt.plot(SalesOrderDetail["DueDate"],SalesOrderDetail["TotalPurchasePrice"], color="black")

plt.plot(SalesOrderDetail["DueDate"],SalesOrderDetail["TotalSalesPrice"], color="green")

plt.title('2013 First & Second Quarter')

plt.xlabel('Sales Order Details')
plt.ylabel('Revenue by Total number of orders')

plt.grid(True)

plt.show()
